In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import warnings
warnings.filterwarnings("ignore")

#constants
DATA_PATH = Path("../dataset/train.csv")
CLEANED_DATA_PATH = Path("../dataset/cleaned_train.csv")

In [ ]:
def load_raw_data():
    """Load raw data from CSV file."""
    df = pd.read_csv(DATA_PATH)
    print(f"Loaded raw data: {df.shape}")
    return df

df_raw = load_raw_data()
df = df_raw.copy()    

Loaded raw data: (1460, 81)
